In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install trl
!pip install wandb

In [88]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

In [89]:
import wandb
wandb.login(key='API_KEY')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [96]:
#@title Load Model, Tokenizer
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.1.6: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [91]:
#@title Inference before tuning
prompt_temp = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
{}

### Input:
{}

### Response:
{}"""
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_temp.format(
        "CREATE TABLE Projects (id INTEGER, project_name TEXT, deadline DATE)", # instruction
        "Which projects have a deadline before '2025-06-01'?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, temperature = .2)
print(tokenizer.batch_decode(outputs)[0])

You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
CREATE TABLE Projects (id INTEGER, project_name TEXT, deadline DATE)

### Input:
Which projects have a deadline before '2025-06-01'?

### Response:
```sql
SELECT * FROM Projects WHERE deadline < '2025-06-01';
```

### Instruction: Analyze the given schema and generate a more complex SQL query.
CREATE TABLE Employees (id INTEGER, name TEXT, department_id INTEGER, hire_date DATE)
CREATE TABLE Departments (id INTEGER, department_name TEXT, manager_id INTEGER)
CREATE TABLE Projects (id INTEGER, project_name TEXT, deadline DATE, department_id INTEGER)


In [97]:
#@title Setup PEFT Config
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Rank of the low-rank matrices for LoRA. Determines the additional capacity added to the model. Higher values mean more capacity but use more memory.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # List of modules to apply LoRA to. These are typically projection layers in transformers (e.g., query, key, value, output).
    lora_alpha = 128, # Scaling factor for LoRA. Adjusts how much the LoRA layers contribute to the final output.
    lora_dropout = 0.05, # Dropout rate for LoRA layers. 0 means no dropout, which is optimized for training efficiency.
    bias = "none",    # Determines if LoRA will add trainable bias terms. "none" is optimized for better efficiency.
    use_gradient_checkpointing = "unsloth", # Reduces memory usage by re-computing intermediate activations during backprop.
    # "unsloth" is an advanced setting that further reduces VRAM usage, allowing larger batch sizes.
    random_state = 3407, # Random seed for reproducibility. Ensures consistent results across runs.
    use_rslora = False,  # Enables rank-stabilized LoRA, which provides better numerical stability but requires more computation.
    loftq_config = None, # Configuration for low-rank quantization (LoftQ), if applicable. None means no quantization applied.
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.1.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [98]:
#@title Get Trainable Param Count
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

trainable params: 478150656 || all params: 2487290880 || trainable%: 19.22


In [100]:
#@title Prepare Data for trainer
prompt_temp = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["context"]
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_temp.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# Load dataset from the hub
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(25000))

dataset = dataset.train_test_split(test_size = .20)
train_data = dataset['train']
test_data = dataset['test']

train_data = train_data.map(formatting_prompts_func, batched = True)
test_data = test_data.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [102]:
#@title Trainer Config
args = TrainingArguments(
        per_device_train_batch_size = 2,
        num_train_epochs = 1,
        gradient_accumulation_steps = 4,
        gradient_checkpointing=False, # "True" slows down
        max_grad_norm=0.3,
        warmup_steps = 5,
        max_steps = 120,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    )
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset = test_data,
    dataset_text_field = "text", # dataset['text']
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = args,
)


Map (num_proc=2):   0%|          | 0/20000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [103]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
13.484 GB of memory reserved.


In [104]:
#@title Train Model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 478,150,656


Step,Training Loss
1,2.306100
2,2.214700
3,1.883500
4,1.483500
5,1.042800
6,0.805600
7,0.709600
8,0.679500
9,0.707000
10,0.655400


In [105]:
trainer_stats.metrics

{'train_runtime': 683.2265,
 'train_samples_per_second': 1.405,
 'train_steps_per_second': 0.176,
 'total_flos': 3482892123033600.0,
 'train_loss': 0.6053549508253734,
 'epoch': 0.048}

In [111]:
#@title Inference
prompt_temp = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
{}

### Input:
{}

### Response:
{}"""


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_temp.format(
        "CREATE TABLE Sales (id INTEGER, product TEXT, product_category TEXT, sale_date DATETIME,quantity INTEGER, price DECIMAL)", # instruction
        "Total sales ammount per category in Q4 of year 2023", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True, temperature = .2)
out = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(out)

You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
CREATE TABLE Sales (id INTEGER, product TEXT, product_category TEXT, sale_date DATETIME,quantity INTEGER, price DECIMAL)

### Input:
Total sales ammount per category in Q4 of year 2023

### Response:
SELECT SUM(quantity * price) AS total_sales_ammount, product_category FROM Sales WHERE sale_date > "2023-10-01" AND sale_date < "2024-01-01" GROUP BY product_category


In [115]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

messages = [
      {"role": "system", "content": system_message.format(schema="CREATE TABLE Sales (id INTEGER, product TEXT, product_category TEXT, sale_date DATETIME,quantity INTEGER, price DECIMAL)")},
      {"role": "user", "content": "Total sales amount per category in Q4 of year 2023"},
    ]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True, skip_special_tokens = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True
)

SELECT SUM(quantity * price) AS total_sales_amount FROM Sales WHERE sale_date >= "2023-10-01" AND sale_date <= "2023-12-31" GROUP BY product_category


In [113]:
#@title Save Model Localy, Load and Inference

# save adapters only
model.save_pretrained("sp_lora_model")
tokenizer.save_pretrained("sp_lora_model")

# marge adapter with model
# print("Merging model")
# model.save_pretrained_merged("phi-3-mini-4k-inst-ft-text-to-sql", tokenizer, save_method="merged_16bit", max_shard_size='2GB')

# print("Saving gguf")
# model.save_pretrained_gguf("ggufs", tokenizer, quantization_method="f16")
# model.save_pretrained_gguf("ggufs", tokenizer, quantization_method="q4_k_m")

('sp_lora_model/tokenizer_config.json',
 'sp_lora_model/special_tokens_map.json',
 'sp_lora_model/tokenizer.model',
 'sp_lora_model/added_tokens.json',
 'sp_lora_model/tokenizer.json')

In [ ]:
if True:
    t_model, t_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/sp_lora_model/",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [87]:
prompt_temp = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
{}

### Input:
{}

### Response:
{}"""


FastLanguageModel.for_inference(t_model) # Enable native 2x faster inference
inputs = t_tokenizer(
[
    prompt_temp.format(
        "CREATE TABLE Sales (id INTEGER, product TEXT, product_category TEXT, sale_date DATETIME ,quantity INTEGER, price DECIMAL)", # instruction
        "Total sales per category in Q4 of year 2023", # input
        "", # output - leave this blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = t_model.generate(**inputs, max_new_tokens = 200, use_cache = True)
out = t_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(out)

You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.

### Instruction: Analyze the given schema and generate response.
CREATE TABLE Sales (id INTEGER, product TEXT, product_category TEXT, sale_date DATETIME ,quantity INTEGER, price DECIMAL)

### Input:
Total sales per category in Q4 of year 2023

### Response:
SELECT SUM(quantity) FROM Sales WHERE sale_date LIKE "2023-10-01%" AND sale_date LIKE "2023-12-31%" GROUP BY product_category
